In [1]:
cd ..

/home/alex/tmp/kaggle-spaceship-titanic


In [2]:


from src.imports import *
from src.transformers import *
from src.vars import *
from src.helpers import *

In [3]:
train = pd.read_csv("data/source/train.csv")

In [4]:
test = pd.read_csv("data/source/test.csv")

In [5]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [6]:
train.shape

(8693, 14)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [8]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object

In [9]:
train.dtypes.value_counts()

object     7
float64    6
bool       1
Name: count, dtype: int64

In [10]:
train.describe(include=np.number).round(2)

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.00,8512.00,8510.00,8485.00,8510.00,8505.00
mean,28.83,224.69,458.08,173.73,311.14,304.85
std,14.49,666.72,1611.49,604.70,1136.71,1145.72
min,0.00,0.00,0.00,0.00,0.00,0.00
25%,19.00,0.00,0.00,0.00,0.00,0.00
50%,27.00,0.00,0.00,0.00,0.00,0.00
75%,38.00,47.00,76.00,27.00,59.00,46.00
max,79.00,14327.00,29813.00,23492.00,22408.00,24133.00


In [11]:
dd = train.describe(exclude=np.number).round(2)
dd

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,VIP,Name,Transported
count,8693,8492,8476,8494,8511,8490,8493,8693
unique,8693,3,2,6560,3,2,8473,2
top,0001_01,Earth,False,G/734/S,TRAPPIST-1e,False,Gollux Reedall,True
freq,1,4602,5439,8,5915,8291,2,4378


In [12]:
train.Transported = train.Transported.astype(int)
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0


In [13]:
X = train.drop("Transported", axis=1)
y = train["Transported"]

In [14]:
dd = dd.T
dd

,count,unique,top,freq
PassengerId,8693,8693,0001_01,1
HomePlanet,8492,3,Earth,4602
CryoSleep,8476,2,False,5439
Cabin,8494,6560,G/734/S,8
Destination,8511,3,TRAPPIST-1e,5915
VIP,8490,2,False,8291
Name,8493,8473,Gollux Reedall,2
Transported,8693,2,True,4378


In [15]:
DISPLAY = False
FRAC = 0.1

In [16]:
if DISPLAY:
    sns.pairplot(train.sample(frac=FRAC), corner=True, hue="Transported")

In [17]:
pipeline = Pipeline(
    [
        ("column_selector", ColumnSelector()),
        ("log_transformer", LogTransformer()),
        ("preprocessor", ColumnTransformer([])),
        ("sampler_1", RandomUnderSampler()),
        ("imputer", KNNImputer()),
        ("scaler", StandardScaler()),
        ("sampler_2", RandomUnderSampler()),
        ("estimator", LogisticRegression()),
    ]
)

In [19]:
param_grid = {
    "log_transformer__threshold": [0.5, 1, 3, 5, 10, 7, 12],
    "preprocessor__transformers": [t0, t1],
    "sampler_1": [RandomUnderSampler(), "passthrough"],
    "scaler": scaler_list,
    "sampler_2": [RandomUnderSampler(), "passthrough"],
    "estimator": [
        LogisticRegression(),
    ],
}

In [25]:
param_grid = {
    "estimator":[ RandomForestClassifier(),],
    "log_transformer__threshold":[ 0.5, 1, 3,],
    "preprocessor__transformers":[ t1,],
    "sampler_1":[ "passthrough",],
    "sampler_2":[ "passthrough",],
    "scaler":[ "passthrough"],
}

In [26]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    # cv=3,
    cv=StratifiedShuffleSplit(n_splits=5, test_size=0.33),  # , random_state=42
    scoring="accuracy",
    n_jobs=-1,
    return_train_score=True,
    verbose=1,
)

In [27]:
grid.fit(X, y)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=5, random_state=None, test_size=0.33,
            train_size=None),
             estimator=Pipeline(steps=[('column_selector', ColumnSelector()),
                                       ('log_transformer', LogTransformer()),
                                       ('preprocessor',
                                        ColumnTransformer(transformers=[])),
                                       ('sampler_1', RandomUnderSampler()),
                                       ('imputer', KNNImputer()),
                                       ('scaler', StandardScaler()),
                                       ('sampler_...
                         'preprocessor__transformers': [[('num', 'passthrough',
                                                          <sklearn.compose._column_transformer.make_column_selector object at 0x7f73d6ab0f70>),
                                                         ('cat',
                                                          OneHotEncoder(),
                                                          <sklearn.compose._column_transformer.make_column_selector object at 0x7f73d6ab2800>)]],
                         'sampler_1': ['passthrough'],
                         'sampler_2': ['passthrough'],
                         'scaler': ['passthrough']},
             return_train_score=True, scoring='accuracy', verbose=1)

In [32]:
res = resultize(grid).head(10)
res

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_estimator,param_log_transformer__threshold,param_preprocessor__transformers,param_sampler_1,param_sampler_2,param_scaler,params,mean_test_score,std_test_score,mean_train_score,std_train_score
1,4.22,0.32,1.10,0.16,RandomForestClassifier(),1,"[(num, passthrough, <sklearn.compose._column_t...",passthrough,passthrough,passthrough,"{'estimator': RandomForestClassifier(), 'log_t...",0.79,0.0,0.94,0.0
0,4.06,0.24,1.01,0.16,RandomForestClassifier(),0.5,"[(num, passthrough, <sklearn.compose._column_t...",passthrough,passthrough,passthrough,"{'estimator': RandomForestClassifier(), 'log_t...",0.78,0.0,0.94,0.0
2,4.17,0.25,1.32,0.40,RandomForestClassifier(),3,"[(num, passthrough, <sklearn.compose._column_t...",passthrough,passthrough,passthrough,"{'estimator': RandomForestClassifier(), 'log_t...",0.78,0.0,0.94,0.0


In [33]:
res_fn = f"./results/{now()}"
txt_fn = f"./models/{now()}"
model_fn = f"./models/{now()}"

In [34]:
res.astype(str).to_csv(res_fn + ".csv", index=False)

with open(txt_fn + ".json", "w") as f:
    txt = str(res.params.iloc[0])
    f.write(txt)


with open(model_fn + ".pk", "wb") as f:
    pickle.dump(grid, f)

In [35]:
res.params.iloc[1]

{'estimator': RandomForestClassifier(),
 'log_transformer__threshold': 0.5,
 'preprocessor__transformers': [('num',
   'passthrough',
   <sklearn.compose._column_transformer.make_column_selector at 0x7f73d6ab0f70>),
  ('cat',
   OneHotEncoder(),
   <sklearn.compose._column_transformer.make_column_selector at 0x7f73d6ab2800>)],
 'sampler_1': 'passthrough',
 'sampler_2': 'passthrough',
 'scaler': 'passthrough'}

In [36]:
res.params.iloc[2]

{'estimator': RandomForestClassifier(),
 'log_transformer__threshold': 3,
 'preprocessor__transformers': [('num',
   'passthrough',
   <sklearn.compose._column_transformer.make_column_selector at 0x7f73d6ab0f70>),
  ('cat',
   OneHotEncoder(),
   <sklearn.compose._column_transformer.make_column_selector at 0x7f73d6ab2800>)],
 'sampler_1': 'passthrough',
 'sampler_2': 'passthrough',
 'scaler': 'passthrough'}

In [51]:
param_grid = {
    "log_transformer__threshold": [3, 5, 10, 7, 12],
    "preprocessor__transformers": [t0, t1],
    "sampler_1": [RandomUnderSampler(), "passthrough"],
    "scaler": ["passthrough", StandardScaler(), Normalizer()],
    "sampler_2": ["passthrough"],
    "estimator": [
        RandomForestClassifier(),
    ],
}

In [ ]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    scoring="accuracy",
    n_jobs=-1,
    return_train_score=True,
    verbose=1,
)

grid.fit(X, y)

In [ ]:
res = resultize(grid).head(10)
res

In [ ]:
grid

In [37]:
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [145]:
preds = pd.DataFrame(zip(test.PassengerId, grid.predict(test)), columns=["PassengerID", "Survived"])
preds_fn = f"./data/preds/{now()}"
preds.to_csv(preds_fn+".csv", index=False)

In [41]:
train

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0


In [47]:
def find_bool(X) : 

    ser = X.iloc[0]


    print(ser)

    print(ser.dtypes)


find_bool(train)

PassengerId             0001_01
HomePlanet               Europa
CryoSleep                 False
Cabin                     B/0/P
Destination         TRAPPIST-1e
Age                        39.0
VIP                       False
RoomService                 0.0
FoodCourt                   0.0
ShoppingMall                0.0
Spa                         0.0
VRDeck                      0.0
Name            Maham Ofracculy
Transported                   0
Name: 0, dtype: object
object


In [49]:
v = train.iloc[0].CryoSleep
v

False

In [50]:
type(v)

bool

In [51]:
np.bool_

numpy.bool_

In [53]:
X.CryoSleep.astype(int)

ValueError: cannot convert float NaN to integer

In [54]:
bool_cols=["CryoSleep", "VIP"]

In [55]:
_X = X.copy()

In [56]:

for col in bool_cols : 
    _X[col] = _X[col].replace({True: 1, False: 0})

In [57]:
_X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,A/98/P,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther
8689,9278_01,Earth,1.0,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley
8690,9279_01,Earth,0.0,G/1500/S,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon
8691,9280_01,Europa,0.0,E/608/S,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre


In [58]:
split_cols=["Cabin"]

In [59]:
for col in split_cols : 
    for i in range(2) : 
        _X[col+_i] = _X[col].apply(lambda i : i.split("/")[i])



TypeError: list indices must be integers or slices, not str

In [61]:
c = X.iloc[0].Cabin
c

'B/0/P'

In [66]:
X.Cabin.isna().mean()

0.02289198205452663

In [67]:
for col in split_cols :
    for i in range(2) : 
        _X[col+_i] = _X[col].apply(lambda i : i.split("/")[i] if isinstance(i, str) else i  )


TypeError: list indices must be integers or slices, not str

In [68]:
for col in split_cols :
    for i in range(2) : 
        _X[col+f"_{i}"] = _X[col].apply(lambda i : i.split("/")[i] if isinstance(i, str) else i  )


TypeError: list indices must be integers or slices, not str

In [90]:
for col in split_cols :
    for i in range(3) : 
        _X[col+f"_{i}"] = _X[col].apply(lambda val : val.split("/")[i] if isinstance(val, str) else val  )
        try : 
            _X[col+f"_{i}"] = _X[col+f"_{i}"].astype(float)
        except Exception as e : 
            pass


In [91]:
split_cols

['Cabin']

In [92]:
_X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin_0,Cabin_1,Cabin_2
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,B,0.0,P
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,F,0.0,S
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,A,0.0,S
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,A,0.0,S
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,F,1.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,A/98/P,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,A,98.0,P
8689,9278_01,Earth,1.0,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,G,1499.0,S
8690,9279_01,Earth,0.0,G/1500/S,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,G,1500.0,S
8691,9280_01,Europa,0.0,E/608/S,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,E,608.0,S


In [93]:
_X.Cabin_0.value_counts()

Cabin_0
F    2794
G    2559
E     876
B     779
C     747
D     478
A     256
T       5
Name: count, dtype: int64

In [94]:
_X.Cabin_1.value_counts()

Cabin_1
82.0      28
86.0      22
19.0      22
56.0      21
176.0     21
          ..
1644.0     1
1515.0     1
1639.0     1
1277.0     1
1894.0     1
Name: count, Length: 1817, dtype: int64

In [96]:
_X.Cabin_2.value_counts()

Cabin_2
S    4288
P    4206
Name: count, dtype: int64

In [97]:
_X.Name

0         Maham Ofracculy
1            Juanna Vines
2           Altark Susent
3            Solam Susent
4       Willy Santantines
              ...        
8688    Gravior Noxnuther
8689      Kurta Mondalley
8690         Fayey Connon
8691     Celeon Hontichre
8692     Propsh Hontichre
Name: Name, Length: 8693, dtype: object

In [101]:
ser = _X.Name.apply(lambda i : len(i.split(" ")) if isinstance(i, str) else i)

In [102]:
ser

0       2.0
1       2.0
2       2.0
3       2.0
4       2.0
       ... 
8688    2.0
8689    2.0
8690    2.0
8691    2.0
8692    2.0
Name: Name, Length: 8693, dtype: float64

In [103]:
ser.value_counts()

Name
2.0    8493
Name: count, dtype: int64

In [104]:
ser =  _X.Name.apply(lambda i : len(i) if isinstance(i, str) else i)


In [105]:
ser

0       15.0
1       12.0
2       13.0
3       12.0
4       17.0
        ... 
8688    17.0
8689    15.0
8690    12.0
8691    16.0
8692    16.0
Name: Name, Length: 8693, dtype: float64

In [107]:
            # _X.Name.apply(lambda i : len(i.split(" ")) if isinstance(i, str) else i) # no because only 2 on train
            _X["_len_Name"] = _X.Name.apply(lambda i : len(i) if isinstance(i, str) else i)
            _X["_FirstName"] = _X.Name.apply(lambda i : i.split(" ")[0] if isinstance(i, str) else i)
            _X["_LastName"] = _X.Name.apply(lambda i : i.split(" ")[1] if isinstance(i, str) else i)



In [108]:
_X

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin_0,Cabin_1,Cabin_2,_len_Name,_FirstName,_LastName
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,B,0.0,P,15.0,Maham,Ofracculy
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,F,0.0,S,12.0,Juanna,Vines
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,A,0.0,S,13.0,Altark,Susent
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,A,0.0,S,12.0,Solam,Susent
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,F,1.0,S,17.0,Willy,Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,A/98/P,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,A,98.0,P,17.0,Gravior,Noxnuther
8689,9278_01,Earth,1.0,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,G,1499.0,S,15.0,Kurta,Mondalley
8690,9279_01,Earth,0.0,G/1500/S,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,G,1500.0,S,12.0,Fayey,Connon
8691,9280_01,Europa,0.0,E/608/S,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,E,608.0,S,16.0,Celeon,Hontichre


In [111]:
_X.isna().mean().round(2)

PassengerId     0.00
HomePlanet      0.02
CryoSleep       0.02
Cabin           0.02
Destination     0.02
Age             0.02
VIP             0.02
RoomService     0.02
FoodCourt       0.02
ShoppingMall    0.02
Spa             0.02
VRDeck          0.02
Name            0.02
Cabin_0         0.02
Cabin_1         0.02
Cabin_2         0.02
_len_Name       0.02
_FirstName      0.02
_LastName       0.02
dtype: float64

In [114]:
tmp = _X.isna().mean(axis=1).round(2)
tmp

0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
8688    0.0
8689    0.0
8690    0.0
8691    0.0
8692    0.0
Length: 8693, dtype: float64

In [116]:
tmp[tmp>0.0].sort_values()

7       0.05
5438    0.05
5434    0.05
5428    0.05
5427    0.05
        ... 
7211    0.32
6057    0.32
4548    0.32
7836    0.42
5806    0.47
Length: 2087, dtype: float64

In [119]:
_X._LastName.value_counts().to_dict()

{'Casonston': 18,
 'Oneiles': 16,
 'Domington': 15,
 'Litthews': 15,
 'Browlerson': 14,
 'Garnes': 14,
 'Cartez': 14,
 'Fulloydez': 14,
 'Hinglendez': 13,
 'Distured': 13,
 'Briggston': 13,
 'Barbes': 13,
 'Moodman': 13,
 'Gibbsonton': 13,
 'Vandan': 12,
 'Fowlesterez': 12,
 'Loway': 12,
 'Vloaf': 12,
 'Gainney': 12,
 'Hanner': 12,
 'Georgasey': 12,
 'Wagnerray': 11,
 'Cofferson': 11,
 'Jenson': 11,
 'Gouldensen': 11,
 'Morelly': 11,
 'Fryersonis': 11,
 'Ingston': 11,
 'Mosteraked': 11,
 'Torrez': 11,
 'Buckentry': 11,
 'Chanan': 11,
 'Crité': 11,
 'Pirejus': 11,
 'Villenson': 11,
 'Ousious': 11,
 'Clugete': 11,
 'Kiling': 11,
 'Ainserfle': 11,
 'Coolerson': 11,
 'Fla': 11,
 'Clemondsey': 11,
 'Belley': 11,
 'Yorkland': 11,
 'Guerson': 11,
 'Willy': 11,
 'Haydenzier': 10,
 'Mclardson': 10,
 'Wilsoney': 10,
 'Minen': 10,
 'Navages': 10,
 'Wartyson': 10,
 'Sageng': 10,
 'Sysilstict': 10,
 'Josey': 10,
 'Braymon': 10,
 'Rilley': 10,
 'Fone': 10,
 'Pread': 10,
 'Birt': 10,
 'Gainebergan': 

In [120]:
[1] if isinstance(i, str) else i)
            family_dict = _X._LastName.value_counts().to_dict()
            _X["_FamilySize"] = _X["_LastName"].replace(family_dict

SyntaxError: unmatched ')' (410698146.py, line 1)

In [122]:
family_dict = _X._LastName.value_counts().to_dict()
_X["_FamilySize"] = _X["_LastName"].replace(family_dict)

In [123]:
_X.drop(columns=[], errors="ignore")

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Cabin_0,Cabin_1,Cabin_2,_len_Name,_FirstName,_LastName,_FamilySize
0,0001_01,Europa,0.0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,B,0.0,P,15.0,Maham,Ofracculy,1.0
1,0002_01,Earth,0.0,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,F,0.0,S,12.0,Juanna,Vines,4.0
2,0003_01,Europa,0.0,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,A,0.0,S,13.0,Altark,Susent,6.0
3,0003_02,Europa,0.0,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,A,0.0,S,12.0,Solam,Susent,6.0
4,0004_01,Earth,0.0,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,F,1.0,S,17.0,Willy,Santantines,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,0.0,A/98/P,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,A,98.0,P,17.0,Gravior,Noxnuther,3.0
8689,9278_01,Earth,1.0,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,G,1499.0,S,15.0,Kurta,Mondalley,2.0
8690,9279_01,Earth,0.0,G/1500/S,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,G,1500.0,S,12.0,Fayey,Connon,6.0
8691,9280_01,Europa,0.0,E/608/S,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,E,608.0,S,16.0,Celeon,Hontichre,6.0


In [125]:
_X.Cabin_0.unique()

array(['B', 'F', 'A', 'G', nan, 'E', 'D', 'C', 'T'], dtype=object)

In [126]:
import string

In [127]:
string.ascii_uppercase

'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [129]:
cab_dict = {i:j for j,i in enumerate(string.ascii_uppercase)}
cab_dict

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9,
 'K': 10,
 'L': 11,
 'M': 12,
 'N': 13,
 'O': 14,
 'P': 15,
 'Q': 16,
 'R': 17,
 'S': 18,
 'T': 19,
 'U': 20,
 'V': 21,
 'W': 22,
 'X': 23,
 'Y': 24,
 'Z': 25}

In [132]:
for col in split_cols :
    for i in range(3) : 
        _X[f"_{col}_{i}"] = _X[col].apply(lambda val : val.split("/")[i].upper() if isinstance(val, str) else val  )
        try : 
            _X[f"_{col}_{i}"] = _X[f"_{col}_{i}"].astype(float)
        except Exception as e : 
            _X[f"_{col}_{i}_int"] =  _X[f"_{col}_{i}"].replace(cab_dict)


In [135]:
_X.iloc[:,  3:]

,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,...,Cabin_2,_len_Name,_FirstName,_LastName,_FamilySize,_Cabin_0,_Cabin_0_int,_Cabin_1,_Cabin_2,_Cabin_2_int
0,B/0/P,TRAPPIST-1e,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,...,P,15.0,Maham,Ofracculy,1.0,B,1.0,0.0,P,15.0
1,F/0/S,TRAPPIST-1e,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,...,S,12.0,Juanna,Vines,4.0,F,5.0,0.0,S,18.0
2,A/0/S,TRAPPIST-1e,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,...,S,13.0,Altark,Susent,6.0,A,0.0,0.0,S,18.0
3,A/0/S,TRAPPIST-1e,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,...,S,12.0,Solam,Susent,6.0,A,0.0,0.0,S,18.0
4,F/1/S,TRAPPIST-1e,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,...,S,17.0,Willy,Santantines,6.0,F,5.0,1.0,S,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,A/98/P,55 Cancri e,41.0,1.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,...,P,17.0,Gravior,Noxnuther,3.0,A,0.0,98.0,P,15.0
8689,G/1499/S,PSO J318.5-22,18.0,0.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,...,S,15.0,Kurta,Mondalley,2.0,G,6.0,1499.0,S,18.0
8690,G/1500/S,TRAPPIST-1e,26.0,0.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,...,S,12.0,Fayey,Connon,6.0,G,6.0,1500.0,S,18.0
8691,E/608/S,55 Cancri e,32.0,0.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,...,S,16.0,Celeon,Hontichre,6.0,E,4.0,608.0,S,18.0
